<a href="https://colab.research.google.com/github/cibercitizen1/cuda_hello/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2wsfbbfz
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2wsfbbfz
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=04a1bb26eabe76c397586d2b66e1eb22c56e96f459a62806569f3f444cd4574e
  Stored in directory: /tmp/pip-ephem-wheel-cache-q555m6na/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu
// -------------------------------------------------------------
// mainHello_1.cu
// -------------------------------------------------------------
#include <cstdio>
#include <iostream>

// -------------------------------------------------------------
// -------------------------------------------------------------
using namespace std;

// -------------------------------------------------------------
// Z -> () -> Z (for a kernel)
//
// [Z] -> () -> [Z] (for all the kernels)
// -------------------------------------------------------------
__global__ void test_kernel(int* p_input, int* p_output) {

  //
  // We wave 1-dim data (i.e. an array)
  // We have arranged one thread for one cell
  // both in the input array and in the output one
  // 
  // Therefore, we have to find out our thread index,
  // which equates to the cell number in the array
  // we have to manipulate
  //
  // The calculation is as follows:
  
  int idx = (blockIdx.x * blockDim.x) + threadIdx.x;

  // blockDim.x: the number of threads in the block for the x index
  // (which in this case is the only one)
  // times
  // blockIdx.x: number of block in for the x index
  // plus
  // threadIdx.x: the thread number within this block
  // Example: if we are the block number 3, each block has 16 threds
  // and the thread number is 7
  // The cell would be 3*16 + 7

  //
  // This is the calculation
  //
  p_output[idx] =  100 + p_input[idx];

}

// -------------------------------------------------------------
// -------------------------------------------------------------
int main() {

  //
  // input and output local arrays
  //
  const int N=1024;
  int numbers[N];
  int results[N];

	 int tam = N * sizeof(int);

  for (int i = 0; i <= N-1; i++) {
	numbers[i] = i;
	results[i] = -1;
  }

 
 

  //
  // get memory in the device
  //
  int* p_in;
  int* p_out;
  //
  cudaMalloc(&p_in, tam);
  cudaMalloc(&p_out, tam);

  //
  // timers, define and start to count
  //
  cudaEvent_t start; 
  cudaEvent_t end;
  cudaEventCreate(&start);
  cudaEventCreate(&end);
  
  cudaEventRecord(start);

  //
  // copy to device
  //
  cudaMemcpy(p_in, numbers, tam, cudaMemcpyHostToDevice);

  dim3 total_blocks( 4 );
  dim3 threads_per_block( N/4 );
  
  // dim3 total_blocks( 1 );
  // dim3 threads_per_block( N );


  //
  // start up the kernel(s)
  //
  test_kernel<<<total_blocks, threads_per_block>>>(p_in, p_out);

  //
  // wait for completion
  //
  cudaEventSynchronize(end);

  //
  // copy from device
  //
  cudaMemcpy(&results[0], p_out,  tam, cudaMemcpyDeviceToHost);
  
  //
  // record end moment, and calculate the elapsed time
  //
  cudaEventRecord(end);
  float time = 0;
  cudaEventElapsedTime(&time, start, end);

  //
  // results
  //

  cout << "results[1] : " << results[1] << endl;

  cout << "results[31] : " << results[31] << endl;
  cout << "results[32] : " << results[32] << endl;

  cout << "results[63] : " << results[63] << endl;
  cout << "results[64] : " << results[64] << endl;
  cout << "results[65] : " << results[65] << endl;
  cout << "results[" << N-1 << "] : " << results[N-1] << endl;


  cout << "start: " << start << endl;
  cout << "end: " << end << endl;
  //cout << (end - start) << endl;
  cout << "The time required : ";
  cout << time << endl;
} // main()
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------

results[1] : 101
results[31] : 131
results[32] : 132
results[63] : 163
results[64] : 164
results[65] : 165
results[1023] : 1123
start: 0x56114121de10
end: 0x56114121dc70
The time required : 0



In [ ]:
!ls


sample_data  src


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%cu
/* This sample queries the properties of the CUDA devices present in the system via CUDA Runtime API. */

// includes, system
#include <stdlib.h>
#include <stdio.h>
#include <string.h>


#include <cuda.h>
#include <cuda_runtime_api.h>

// includes, project
// #include <cutil.h>

////////////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////////////
int
main( int argc, char** argv) 
{
    printf("CUDA Device Query (Runtime API) version (CUDART static linking)\n");

    int deviceCount = 0;

	if (cudaGetDeviceCount(&deviceCount) != cudaSuccess) {
		printf("cudaGetDeviceCount failed! CUDA Driver and Runtime version may be mismatched.\n");
		printf("\nTest FAILED!\n");
		return 0;
	}

    // This function call returns 0 if there are no CUDA capable devices.
    if (deviceCount == 0)
        printf("There is no device supporting CUDA\n");

    int dev;
    for (dev = 0; dev < deviceCount; ++dev) {
        cudaDeviceProp deviceProp;
        cudaGetDeviceProperties(&deviceProp, dev);

        if (dev == 0) {
			// This function call returns 9999 for both major & minor fields, if no CUDA capable devices are present
            if (deviceProp.major == 9999 && deviceProp.minor == 9999)
                printf("There is no device supporting CUDA.\n");
            else if (deviceCount == 1)
                printf("There is 1 device supporting CUDA\n");
            else
                printf("There are %d devices supporting CUDA\n", deviceCount);
        }
        printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);
    #if CUDART_VERSION >= 2020
		int driverVersion = 0, runtimeVersion = 0;
		cudaDriverGetVersion(&driverVersion);
		printf("  CUDA Driver Version:                           %d.%d\n", driverVersion/1000, driverVersion%100);
		cudaRuntimeGetVersion(&runtimeVersion);
		printf("  CUDA Runtime Version:                          %d.%d\n", runtimeVersion/1000, runtimeVersion%100);
    #endif

        printf("  CUDA Capability Major revision number:         %d\n", deviceProp.major);
        printf("  CUDA Capability Minor revision number:         %d\n", deviceProp.minor);

		printf("  Total amount of global memory:                 %u bytes\n", deviceProp.totalGlobalMem);
    #if CUDART_VERSION >= 2000
        printf("  Number of multiprocessors:                     %d\n", deviceProp.multiProcessorCount);
        printf("  Number of cores:                               %d\n", 8 * deviceProp.multiProcessorCount);
    #endif
        printf("  Total amount of constant memory:               %u bytes\n", deviceProp.totalConstMem); 
        printf("  Total amount of shared memory per block:       %u bytes\n", deviceProp.sharedMemPerBlock);
        printf("  Total number of registers available per block: %d\n", deviceProp.regsPerBlock);
        printf("  Warp size:                                     %d\n", deviceProp.warpSize);
        printf("  Maximum number of threads per block:           %d\n", deviceProp.maxThreadsPerBlock);
        printf("  Maximum sizes of each dimension of a block:    %d x %d x %d\n",
               deviceProp.maxThreadsDim[0],
               deviceProp.maxThreadsDim[1],
               deviceProp.maxThreadsDim[2]);
        printf("  Maximum sizes of each dimension of a grid:     %d x %d x %d\n",
               deviceProp.maxGridSize[0],
               deviceProp.maxGridSize[1],
               deviceProp.maxGridSize[2]);
        printf("  Maximum memory pitch:                          %u bytes\n", deviceProp.memPitch);
        printf("  Texture alignment:                             %u bytes\n", deviceProp.textureAlignment);
        printf("  Clock rate:                                    %.2f GHz\n", deviceProp.clockRate * 1e-6f);
    #if CUDART_VERSION >= 2000
        printf("  Concurrent copy and execution:                 %s\n", deviceProp.deviceOverlap ? "Yes" : "No");
    #endif
    #if CUDART_VERSION >= 2020
        printf("  Run time limit on kernels:                     %s\n", deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
        printf("  Integrated:                                    %s\n", deviceProp.integrated ? "Yes" : "No");
        printf("  Support host page-locked memory mapping:       %s\n", deviceProp.canMapHostMemory ? "Yes" : "No");
        printf("  Compute mode:                                  %s\n", deviceProp.computeMode == cudaComputeModeDefault ?
			                                                            "Default (multiple host threads can use this device simultaneously)" :
		                                                                deviceProp.computeMode == cudaComputeModeExclusive ?
																		"Exclusive (only one host thread at a time can use this device)" :
		                                                                deviceProp.computeMode == cudaComputeModeProhibited ?
																		"Prohibited (no host thread can use this device)" :
																		"Unknown");
    #endif
	}
    printf("\nTest PASSED\n");

    return 0;
}


CUDA Device Query (Runtime API) version (CUDART static linking)
There is 1 device supporting CUDA

Device 0: "Tesla T4"
  CUDA Driver Version:                           11.60
  CUDA Runtime Version:                          11.20
  CUDA Capability Major revision number:         7
  CUDA Capability Minor revision number:         5
  Total amount of global memory:                 2958950400 bytes
  Number of multiprocessors:                     40
  Number of cores:                               320
  Total amount of constant memory:               65536 bytes
  Total amount of shared memory per block:       49152 bytes
  Total number of registers available per block: 65536
  Warp size:                                     32
  Maximum number of threads per block:           1024
  Maximum sizes of each dimension of a block:    1024 x 1024 x 64
  Maximum sizes of each dimension of a grid:     2147483647 x 65535 x 65535
  Maximum memory pitch:                          2147483647 bytes
  Text

In [22]:
%%cu
// -------------------------------------------------------------
// mainHello_2.cu
// -------------------------------------------------------------
#include <cstdio>
#include <iostream>


// -------------------------------------------------------------
// -------------------------------------------------------------
using namespace std;

// -------------------------------------------------------------
// Shift to the left the contents of the array 
// -------------------------------------------------------------
__global__ void test_kernel( int* p_data, const int N ) {

  int aux;

  // find out my id
  int idx = (blockIdx.x * blockDim.x) + threadIdx.x;

  // let's slow down some threads
  if ( idx % 3 == 0) {
    // read "my own" element
    aux = p_data[idx]; 

    // just to keep the thread busy
    // and have the [idx] element with
    // a wrong value during this lapse
    for (int i=1; i<=10*10*10*10; i++ ) {
      p_data[idx] = -1234;
      p_data[idx] = floorf( sinf( i*i ) );
    } // for

    // finaly we put back the original value of [idx]
    p_data[idx] = aux; 
  } // if

  // supposedly, all threads stop here
  // so that when they pass this point
  // the array is as before
  __syncthreads(); 
  
  // caution: __syncthreads only syncs threads
  // corresponding to the same block
  // Thus, if there are several blocks one
  // thread per block migh read a wrong value
  // (i.e. not the original element but
  // the new one if already set by its neighbour)
  
  //
  // shift to the left in two steps: 1 read 2 write
  //
  // 1 read
  aux = p_data[ (idx+1) % N ];
  
  //
  // each thread should wait until the rest 
  // have read the value of its neighbour element.
  //
  __syncthreads(); 

  // 2 write
  p_data[idx] = aux;

} // ()

// -------------------------------------------------------------
// -------------------------------------------------------------
int main() {

  //
  // input and output local arrays
  //
  const int N=1024;
  int numbers[N];

	 int tam = N * sizeof(int);

  for (int i = 0; i <= N-1; i++) {
	  numbers[i] = i;
  }

  //
  // get memory in the device
  //
  int* p_data;
  //
  cudaMalloc(&p_data, tam);

  //
  // copy to device
  //
  cudaMemcpy(p_data, numbers, tam, cudaMemcpyHostToDevice);

  // we want 1 thread per element => N threads
  const int NUM_BLOCKS = 8;
  dim3 total_blocks( NUM_BLOCKS );
  dim3 threads_per_block( N / NUM_BLOCKS );
  
  //
  // timer
  //
  cudaEvent_t end;
  cudaEventCreate(&end);

  //
  // start up the kernel(s)
  //
  test_kernel<<<total_blocks, threads_per_block>>>(p_data, N);

  //
  // wait for completion
  //
  cudaEventSynchronize(end);

  //
  // copy from device
  //
  cudaMemcpy(&numbers[0], p_data,  tam, cudaMemcpyDeviceToHost);
  
  //
  // results
  //

  cout << "numbers[0] : " << numbers[0] << endl;
  cout << "numbers[1] : " << numbers[1] << endl;

  cout << "numbers[12] : " << numbers[12] << endl;
  cout << "numbers[13] : " << numbers[13] << endl;
  cout << "numbers[14] : " << numbers[14] << endl;

  cout << "numbers[31] : " << numbers[31] << endl;
  cout << "numbers[32] : " << numbers[32] << endl;
  cout << "numbers[33] : " << numbers[33] << endl;

  cout << "numbers[" << N-1 << "] : " << numbers[N-1] << endl;

  for (int i=0; i<=N-1; i++) {
     if (numbers[i] != (i+1) % N ) {
      cout << "element " << i << " is wrong" << endl;
      cout << "numbers[" << i << "] : " << numbers[i] << endl;
     }
  }

} // main()
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------

numbers[0] : 1
numbers[1] : 2
numbers[12] : 13
numbers[13] : 14
numbers[14] : 15
numbers[31] : 32
numbers[32] : 33
numbers[33] : 34
numbers[1023] : 0
element 127 is wrong
numbers[127] : 129
element 255 is wrong
numbers[255] : 257
element 383 is wrong
numbers[383] : -1234
element 511 is wrong
numbers[511] : 513
element 639 is wrong
numbers[639] : 641
element 767 is wrong
numbers[767] : -1234
element 895 is wrong
numbers[895] : 897



In [61]:
%%cu
// -*- mode: c++ -*-
// ===================================================================
// mainGeneral.cu
// ===================================================================

#include <stdio.h>
#include <assert.h>
//#include <cuda.h>

// #include <typeinfo>

// ===================================================================
// ===================================================================

// The number of threads per block is influenced by how many
// local memory a kernel uses. The more memory used, the lesser
// number of threads a block can have.
//
// We define the block's number of threads in 2D for we will be
// operate on 2D data.

const unsigned int BLOCK_SIDE = 8;
dim3 THREADS_PER_BLOCK( BLOCK_SIDE, BLOCK_SIDE );

// ===================================================================
// ===================================================================
// As long as  we are using generated data (i.e. not read from a file),
// we choose its size here.
const unsigned int WIDTH_COLUMNS_X = 128; //512;
const unsigned int HEIGHT_ROWS_Y = 128; //512;

// type of the elements on the 2D input data
typedef float Element_Type;

// type of the results
typedef float Result_Type;

// ===================================================================
// ===================================================================
void check_cuda_call( const char * msg = "" ) {
  
  cudaError_t error = cudaGetLastError();
  
  if ( error != cudaSuccess ) {
      
   printf( " check_cuda_call: failed %s, reason: %s \n", msg, cudaGetErrorString( error ));
   exit(0);
    assert( error == cudaSuccess );
  } 
} // ()

// ===================================================================
// utility for malloc()
// ===================================================================
//class Malloc_Error {};

// ===================================================================
template<typename T>
T my_malloc( const long unsigned size )
// spec. no longer needed: throw ( Malloc_Error )
{
  void * ptr = malloc( size );

  assert( ptr != nullptr && "my_malloc failed" );

  return static_cast<T>( ptr );
} // ()

// ===================================================================
/*
template<typename T, unsigned int NUM_ROWS, unsigned int NUM_COLUMNS>
auto my_malloc_2D_OK( ) {
  auto ptr = new T[NUM_ROWS][NUM_COLUMNS];
  if ( ptr == nullptr ) {
	throw Malloc_Error {};
  }
  return ptr;
} // ()
*/

// ===================================================================
// Let's use cudaMallocHost()
// which gets "pinned memory" in the CPU for us.
// I guess that means that the memory is aligned so that transfers
// from and to the GPU are faster.
template<typename T>
T * my_malloc_2D( unsigned int NUM_ROWS, unsigned int NUM_COLUMNS) {
  
  //
  // compute the size required
  //
  size_t size = NUM_ROWS * NUM_COLUMNS * sizeof( T );

  //printf( "my_malloc_2D(): rows=%d columns=%d, size=%d\n", NUM_ROWS, NUM_COLUMNS, size );

  //
  // malloc
  //
  T* ptr = nullptr;
  cudaMallocHost( & ptr, size );

  check_cuda_call( "my_malloc_2D(): cudaMallocHost()" );
  
  //
  // make sure we've got memory
  //
  assert( ptr != nullptr && "my_malloc_2D failed" );

  //return ( T * [] ) ptr;
  return ptr;
} // ()

/*
  //auto kk = new int [10][20];
  // OK int (* kk)[20] = new int [10][20];
  int (* kk)[20] = new int [10][20];
  kk[9][2] = 13;
*/

// ===================================================================
// Utility class for allocating memory both on the device
// and on the host.
// ===================================================================
template<typename T>
class Results_Holder {
private:
  const unsigned int NUM_ROWS;
  const unsigned int NUM_COLUMNS;
public:
  T * results_on_host;
  T * results_on_device;

  // -----------------------------------------------------------------
  // Used to access to the correct row of results_on_host.
  // Column dimension is required to get the correct one.
  // Because a pointer is returned, [] can be chained:
  // Example:
  // results[10][15]
  // -----------------------------------------------------------------
  const T & operator()( unsigned int row, unsigned int col ) {
	return  results_on_host[ (row * NUM_COLUMNS) + col ];
  } // ()

  // -----------------------------------------------------------------
  // destructor
  // -----------------------------------------------------------------
  ~Results_Holder( ) {
	cudaFree( results_on_host );
	cudaFree( results_on_device );
	printf( " results memory (host and device) freed \n" );
  } // ()

  // -----------------------------------------------------------------
  // constructor
  // -----------------------------------------------------------------
  Results_Holder( unsigned int num_rows, unsigned int num_columns )
	: NUM_ROWS( num_rows ), NUM_COLUMNS( num_columns )
  {
	//
	// Get memory on the host.
	//
	results_on_host = my_malloc_2D< T >( NUM_ROWS, NUM_COLUMNS );
  
	//
	// Get memory on the device. Regular memory I guess, i.e. not a texture.
	//
	// Right now: I don't the differences between cudaMalloc and
	// cudaMallocManaged.
	//
	cudaMallocManaged( & results_on_device,
					   NUM_ROWS * NUM_COLUMNS * sizeof( T )
					   );

	check_cuda_call( " Results_Holder: cudaMallocManaged()" );
  } // ()

  // -----------------------------------------------------------------
  // -----------------------------------------------------------------
  void copy_results_device_to_host() {
	cudaMemcpy( results_on_host,
				results_on_device,
				NUM_ROWS * NUM_COLUMNS * sizeof( T ),
				cudaMemcpyDeviceToHost );
	check_cuda_call( " copy_results_device_to_host " );
  } // ()

  // -----------------------------------------------------------------
  // -----------------------------------------------------------------
}; // class

// ===================================================================
// Utility class for allocating memory on the device 
// binding it to a texture and copying the input data on the host
// to it.
// ===================================================================
template<typename T>
class Texture_Memory_Holder {
private:
  const unsigned int NUM_ROWS;
  const unsigned int NUM_COLUMNS;
public:
  
  cudaChannelFormatDesc channel_desc;

  T* data_on_device;
  
  cudaTextureObject_t texture;
  cudaResourceDesc resource_desc;
  cudaTextureDesc texture_desc;

  // -----------------------------------------------------------------
  // destructor
  // -----------------------------------------------------------------
  ~Texture_Memory_Holder( ) {
	cudaFree( data_on_device );
	cudaDestroyTextureObject( texture );
	printf( " data_on_device and texture memory freed \n" );
  } // ()

  // -----------------------------------------------------------------
  // constructor
  // -----------------------------------------------------------------
  Texture_Memory_Holder(
					   Element_Type (*p_data)[],
					   unsigned int num_rows,
					   unsigned int num_columns
					   )
	: NUM_ROWS( num_rows ), NUM_COLUMNS( num_columns )
  {

	//
	// get memory on the GPU to place our data
	//

	size_t total_size = NUM_ROWS * NUM_COLUMNS * sizeof( T ); 

	cudaMalloc( & data_on_device, total_size );

	check_cuda_call( " Texture_Memory_Holder: cudaMalloc() " );

  printf( " Texture_Memory_Holder: element_type_size=%d, rows=%d, cols=%d, total_size=%zu\n", 
        sizeof( Element_Type ),
         NUM_ROWS, NUM_COLUMNS, total_size );

	//
	// copy the data from here to the memory on the device
	//
	cudaMemcpy( data_on_device, // destination
						 p_data, // source
             total_size, // size
						 cudaMemcpyHostToDevice );

  check_cuda_call( " Texture_Memory_Holder: cudaMemcpy() " );

	//
	// create a channel.  What is this for?
	//
	channel_desc =
    cudaCreateChannelDesc< Element_Type >();
	  //cudaCreateChannelDesc( 32, 0, 0, 0, cudaChannelFormatKindFloat );

	check_cuda_call( " Texture_Memory_Holder: cudaCreateChannelDesc() " );

	//
	// create and configure a texture
	//
	memset( & resource_desc, 0, sizeof( cudaResourceDesc ) );

	resource_desc.resType = cudaResourceTypePitch2D;

  resource_desc.res.pitch2D.devPtr = data_on_device;

  resource_desc.res.pitch2D.width = NUM_COLUMNS;
  resource_desc.res.pitch2D.height = NUM_ROWS;

  resource_desc.res.pitch2D.desc = channel_desc;

  resource_desc.res.pitch2D.pitchInBytes = NUM_COLUMNS * sizeof( Element_Type );

  //
  //
  //
	memset( & texture_desc, 0, sizeof( cudaTextureDesc ) );

	// Last time I set this. Why?
	//texture_desc.normalizedCoords = false;  
	//texture_desc.readMode = cudaReadModeElementType;

	// Here it is where the texture is actually created
	cudaCreateTextureObject( & texture,
							 & resource_desc,
							 & texture_desc,
							 nullptr );

	check_cuda_call( " Texture_Memory_Holder:  cudaCreateTextureObject() " );
  } // ()

  // -----------------------------------------------------------------
  // -----------------------------------------------------------------
};

// ===================================================================
//
// kernel
//
// ===================================================================
__global__ void test_kernel_1( Result_Type * p_results,
							   unsigned int width,
							   unsigned int height,
							   cudaTextureObject_t in_data_texture
							   ) {

  unsigned int x_column = (blockIdx.x * blockDim.x) + threadIdx.x;
  unsigned int y_row = (blockIdx.y * blockDim.y) + threadIdx.y;

  Element_Type input_val =
	tex2D<Element_Type>( in_data_texture, x_column+0.5f, y_row+0.5f );

  p_results[ (width * y_row) + x_column ] = -input_val;
	
} // ()

// ===================================================================
// ===================================================================
template<typename T>
auto make_up_some_data(
					   unsigned int NUM_ROWS,
					   unsigned int NUM_COLUMNS
					   ) {
  
  //
  // Malloc on the host
  //
  T (*p_data)[NUM_COLUMNS] =
	(T (*)[NUM_COLUMNS]) my_malloc_2D<T>( NUM_ROWS, NUM_COLUMNS );
  // The casting to T (*)[NUM_COLUMNS] is for
  // using 2D indexing (i.e [i][j]) instead
  // of doing the maths [ i*num_cols + j ] ourselves.

  printf( " got the memory for input data \n" );

  //
  // Fill in the data
  // Each element is a float: row.col. Ex. 10.15 is row 10, col 15
  //
  for ( unsigned int row = 0; row < NUM_ROWS; row++ ) {
	//printf( " row %d \n", row );
	for ( unsigned int col = 0; col < NUM_COLUMNS; col++ ) {
	  p_data[ row ][ col ] = row + col/1000.0;
	} // for
  } // for

  //printf( " %f \n", p_data2[ 10*WIDTH_COLUMNS_X + 15 ] );
  printf( " %f \n", p_data[10][15] );

  
  //
  //
  //
  return (T (*)[]) p_data;

} // ()

// ===================================================================
//
// main
//
// ===================================================================
int main( int n_args, char * args[] ) {

  printf( " starting \n" );

  // .................................................................
  // Create the input data for the kernels
  // to compute something on it
  // .................................................................
  auto p_data =
	make_up_some_data<Element_Type>( HEIGHT_ROWS_Y, WIDTH_COLUMNS_X );
  
  printf( " input data generated \n" );
 
  // .................................................................
  // Copy the input data to the device, in a texture
  // .................................................................
  Texture_Memory_Holder<Element_Type>
	data_in_texture( p_data, HEIGHT_ROWS_Y, WIDTH_COLUMNS_X ); 

  printf( " placed input data in device memory, bound to a texture \n" );
					   
  // .................................................................
  // Get memory to hold the results (on the GPU and on the CPU)
  // Let's suppose that we get a result for each input element.
  // .................................................................
  Results_Holder<Result_Type>
	results( HEIGHT_ROWS_Y, WIDTH_COLUMNS_X );

  printf( " got data for the results \n" );
				   
  // .................................................................
  // set up the launch of kernels
  // .................................................................
  // Number of blocks we need considering a thread per element (pixel)
  // in the 2D data
  // Defined in 2D.
  //
  dim3 NUM_BLOCKS( WIDTH_COLUMNS_X / THREADS_PER_BLOCK.x,
				   HEIGHT_ROWS_Y / THREADS_PER_BLOCK.y );

  // .................................................................
  // Launch the kernel
  // .................................................................
  printf( " launching kernels \n" );
  test_kernel_1<<< NUM_BLOCKS, THREADS_PER_BLOCK >>>
	(
	 results.results_on_device,
	 WIDTH_COLUMNS_X,
	 HEIGHT_ROWS_Y,
	 data_in_texture.texture
	 );
  

  // .................................................................
  // wait
  // .................................................................
  cudaDeviceSynchronize();

  check_cuda_call( " kernels done\n" );
  
  printf( " kernels done \n" );

  // .................................................................
  // Copy results from memory device
  // .................................................................
  results.copy_results_device_to_host();

  // show sth. to check if the kernel has done something
  printf( " %f \n", results(10, 15) );
  printf( " %f \n", results(25, 40) );
  printf( " %f \n", results(5, 5) );

  //printf( " %f \n", results.results_on_host[10][15] );

  // .................................................................
  // free the memory
  // .................................................................
  // Memory on the host (CPU)
  cudaFree( p_data );
  
  // The memory for the results ( host and device ) is freed by
  // the destructor of results

  // The memory on the texture is freed by
  // the destructor of data_texture

  // .................................................................
  // .................................................................
  printf( " all done \n" );
} // ()

// ===================================================================
// ===================================================================
// ===================================================================
// ===================================================================


 starting 
 got the memory for input data 
 10.015000 
 input data generated 
 Texture_Memory_Holder: element_type_size=4, rows=128, cols=128, total_size=65536
 placed input data in device memory, bound to a texture 
 got data for the results 
 launching kernels 
 kernels done 
 -10.015000 
 -25.040001 
 -5.005000 
 all done 
 results memory (host and device) freed 
 data_on_device and texture memory freed 



In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-q9u_lr7e
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-q9u_lr7e
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=3f49f89767c856de0ee39831ce25691a68193637214915b789dabc1ba0a11111
  Stored in directory: /tmp/pip-ephem-wheel-cache-axbyw1p2/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [45]:
%%cu
#include <stdio.h>
#include <stdint.h>

typedef uint8_t mt;  // use an integer type

__global__ void kernel(cudaTextureObject_t tex)
{
  int x = threadIdx.x;
  int y = threadIdx.y;
  mt val = tex2D<mt>(tex, x, y);
  printf("%d, ", val);
}

int main(int argc, char **argv)
{
  cudaDeviceProp prop;
  cudaGetDeviceProperties(&prop, 0);
  printf("texturePitchAlignment: %lu\n", prop.texturePitchAlignment);
  cudaTextureObject_t tex;
  const int num_rows = 4;
  const int num_cols = prop.texturePitchAlignment*2; // should be able to use a different multiplier here
  const int ts = num_cols*num_rows;
  const int ds = ts*sizeof(mt);
  mt dataIn[ds];
  for (int i = 0; i < ts; i++) dataIn[i] = i;
  mt* dataDev = 0;
  cudaMalloc((void**)&dataDev, ds);
  cudaMemcpy(dataDev, dataIn, ds, cudaMemcpyHostToDevice);
  struct cudaResourceDesc resDesc;
  memset(&resDesc, 0, sizeof(resDesc));
  resDesc.resType = cudaResourceTypePitch2D;
  resDesc.res.pitch2D.devPtr = dataDev;
  resDesc.res.pitch2D.width = num_cols;
  resDesc.res.pitch2D.height = num_rows;
  resDesc.res.pitch2D.desc = cudaCreateChannelDesc<mt>();
  resDesc.res.pitch2D.pitchInBytes = num_cols*sizeof(mt);
  struct cudaTextureDesc texDesc;
  memset(&texDesc, 0, sizeof(texDesc));
  cudaCreateTextureObject(&tex, &resDesc, &texDesc, NULL);
  dim3 threads(4, 4);
  kernel<<<1, threads>>>(tex);
  cudaDeviceSynchronize();
  printf("\n");
  return 0;
}

texturePitchAlignment: 32
0, 1, 2, 3, 64, 65, 66, 67, 128, 129, 130, 131, 192, 193, 194, 195, 

